<a href="https://colab.research.google.com/github/LukeSchreiber/FastAI-Projects/blob/main/Lesson3Project_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Welcome to my lesson 3 project! in this one i made a image classifer that can recognize different shoes! this build upon week 1 and 2 with the addition of learning rates.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path
BASE = Path('/content/drive/MyDrive/shoe_project')
DATA_DIR = BASE/'data'
DATA_DIR.mkdir(parents=True, exist_ok=True)
print('Saving to:', DATA_DIR)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saving to: /content/drive/MyDrive/shoe_project/data


Mount the drive and have the project folders in there aswell

In [ ]:
import os, shutil
from google.colab import files

if not os.path.exists('/root/.kaggle/kaggle.json'):
    uploaded = files.upload()
    os.makedirs('/root/.kaggle', exist_ok=True)
    shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')
    !chmod 600 /root/.kaggle/kaggle.json

!pip -q install kaggle


Saving kaggle.json to kaggle.json


Upload your kaggle.json to enable dataset downloads from Kaggle.

In [ ]:
!kaggle datasets download -d noobyogi0100/shoe-dataset -p "$DATA_DIR" -q
!unzip -q "$DATA_DIR/shoe-dataset.zip" -d "$DATA_DIR"

import glob, os
candidates = [p for p in glob.glob(str(DATA_DIR/'*')) if os.path.isdir(p)]
print('Found:', candidates)


Dataset URL: https://www.kaggle.com/datasets/noobyogi0100/shoe-dataset
License(s): CC0-1.0
replace /content/drive/MyDrive/shoe_project/data/shoeTypeClassifierDataset/training/boots/image1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

Grab the shoe dataset from Kaggle, extract it, and auto-detect the folder structure.

In [ ]:
from fastai.vision.all import *
from pathlib import Path

BASE_NEST = Path(sorted(candidates)[0]) if candidates else DATA_DIR
DATA = BASE_NEST/'train' if (BASE_NEST/'train').exists() else BASE_NEST

print('Using DATA =', DATA)
for p in (DATA.ls().sorted()):
    if p.is_dir(): print(f'{p.name:12s}', len(get_image_files(p)))


Point to the train folder if it exists, otherwise use the base folder. Print classes and counts.

In [ ]:
failed = verify_images(get_image_files(DATA)); failed.map(Path.unlink); print('removed:', len(failed))

dblock = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(256, method='pad'),
    batch_tfms=aug_transforms(size=224, min_scale=0.6, max_rotate=15, p_lighting=0.9)
)
dls = dblock.dataloaders(DATA, bs=32)
dls.show_batch(max_n=12)
print('classes:', dls.vocab)


get the failed images out. create the datablock and datalaoder. show 12 examples. print the classes/groups that were going to look for

In [ ]:
learn = vision_learner(dls, resnet34, metrics=accuracy)
res = learn.lr_find()

lr = res.valley or res.minimum
print("Using LR:", lr)

# Train
learn.fine_tune(5, base_lr=lr)


find the learning rate. then print out it. then show the graph. then train the model

In [ ]:
learn.export('model.pkl')


export model so we can actually use it

In [ ]:
!pip -q install gradio

from fastai.vision.all import *
import gradio as gr

#load model.pkl(the exported leaner)
learn = load_learner('/content/model.pkl')

#Predict!
def predict(img: PILImage):
    label, idx, probs = learn.predict(img)
    vocab = learn.dls.vocab
    topk = min(3, len(vocab))
    vals, inds = probs.topk(topk)
    return {str(vocab[i]): float(v) for v, i in zip(vals, inds)}

#gradio UI
classes_text = ", ".join(map(str, learn.dls.vocab))
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="Upload an image"),
    outputs=gr.Label(num_top_classes=3),
    title="Image Classifier",
    description=(
        "Upload an image. The model was trained on this project’s dataset.\n"
        "Out-of-distribution images may be misclassified.\n"
        f"Classes: {classes_text}"
    ),
)

demo.launch(share=True)  # set share=False if you don’t need a public link


made a UI so you can actually use the model